In [7]:
import pandas as pd

In [12]:
stocks = pd.read_csv('stocks_clean/AAPL.csv')

In [13]:
stocks.head()

,ticker,per,date,time,open,high,low,close,vol
0,AAPL,60,20150102,180000,111.390,111.44,108.650,109.5000,9886884
1,AAPL,60,20150102,190000,109.490,109.71,107.350,108.2850,12628997
2,AAPL,60,20150102,200000,108.290,108.79,107.850,108.7761,6291776
3,AAPL,60,20150102,210000,108.770,109.59,108.755,109.3550,5049924
4,AAPL,60,20150102,220000,109.355,109.47,108.750,108.9900,3663606
